In [ ]:
import pydicom
from PIL import Image
import numpy as np
import os
import glob


dicom_folders = glob.glob('./DICOM/' + '*')
save_dir = './Dataset2png/'


i = 0
for dcm in dcm_folders:
    ds = pydicom.read_file(dcm[0])
    dcm_img = ds.pixel_array


    """ Preprocessing """
    # Applying Rescaling
    if ds.get("RescaleIntercept"):
      intercept = ds.RescaleIntercept
      slope = ds.RescaleSlope
      dcm_img = dcm_img.astype(np.float64) * slope + intercept


    # Applying Value of Interest(VOI) Look-up table
    dcm_img = pydicom.pixel_data_handlers.apply_voi_lut(dcm_img, ds)


    # Complement Check
    if ds[0x0028, 0x0004].value != "MONOCHROME2" :
        norm_img = 1 - (dcm_img - np.min(dcm_img)) / (np.max(dcm_img) - np.min(dcm_img))
    else:
        norm_img = (dcm_img - np.min(dcm_img)) / (np.max(dcm_img) - np.min(dcm_img))


    # Restore to a range of 0~255 pixels
    normalized_dcm = np.uint8(norm_img*255)


    """ Save """
    Image.fromarray(normalized_dcm).save(save_dir + str(xlsx_ID[i]) + ".png")
    i +=1

print("*----- Save Done -----*")